In [104]:
import pandas as pd
import numpy as np

from pathlib import Path
from functools import reduce

In [105]:
df_final = pd.read_csv('./final_dfs/df_final_without_labor_with_clusters.csv', converters={'ags2': str, 'ags5': str})
company_changes_date = pd.read_csv('./final_dfs/company_changes_by_date.csv', converters={'ags2': str, 'ags5': str, 'date':str})
labor_market_by_date = pd.read_csv('./final_dfs/labor_market_by_date.csv', converters={'ags2': str, 'ags5': str, 'date':str})
#labor_market_region = pd.read_csv('./final_dfs/labor_market_region.csv', converters={'ags2': str, 'ags5': str, 'date':str})
funding_eligibility = pd.read_csv('./final_dfs/funding_eligibility.csv', converters={'ags2': str, 'ags5': str, 'date':str})
categorical_columns = pd.read_csv('./final_dfs/categorical_columns.csv', converters={'ags2': str, 'ags5': str, 'date':str})

In [106]:
for col in ['support_area_status','eligible_area']:
    funding_eligibility[col] = funding_eligibility[col].astype('category')
    
    
for col in ['labor_market_region',
       'growing_/_shrinking_circles', 'labor_market_type',
       'grw_funding_framework',
       'settlement_structure_type_of_labor_market_region',
       'room_type_location', 'district_settlement_structure',
       'type_of_settlement_structure', 'urban_/_rural', 'metropolitan_region',
       'metropolitan_area', 'east_west', 'border_proximity']:
    categorical_columns[col] = categorical_columns[col].astype('category')
    

In [107]:
df_final_stationery = df_final.merge(categorical_columns, left_on=['ags5'], right_on=['ags5'])
df_final_stationery = df_final_stationery.merge(funding_eligibility, left_on=['ags5'], right_on=['ags5'])
df_final_stationery = df_final_stationery.drop(['Unnamed: 0_x', 'data_index'], 1)
df_final_stationery

,cluster,kreis,ags5,ags2,supermarkets_population,supermarkets_average_distance,public_transport_availability,average_distance_bus_stop,average_distance_train_station,average_distance_public_transport,...,district_settlement_structure,type_of_settlement_structure,urban_/_rural,metropolitan_region,metropolitan_area,east_west,border_proximity,Unnamed: 0,support_area_status,eligible_area
0,2,Regionalverband Saarbrücken,10041,10,76,734,52,260,1804,255,...,2,1,1,99,99,1,1,318,D/N,1
1,0,Landkreis Merzig-Wadern,10042,10,42,1733,30,269,3777,263,...,2,1,1,99,99,1,1,319,N,0
2,0,Landkreis Neunkirchen,10043,10,76,799,38,229,1762,226,...,2,1,1,99,99,1,2,320,C/D,1
3,0,Landkreis Saarlouis,10044,10,66,1083,37,284,3004,284,...,2,1,1,99,99,1,1,321,C/D,1
4,0,Saarpfalz-Kreis,10045,10,58,1177,38,231,2305,230,...,2,1,1,99,99,1,1,322,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,0,Saalfeld-Rudolstadt,16073,16,62,1423,37,341,2492,322,...,4,2,2,99,99,2,0,396,C*,1
79,0,Saale-Holzland-Kreis,16074,16,54,1677,38,343,3419,340,...,3,2,2,99,99,2,0,397,C*,1
80,0,Saale-Orla-Kreis,16075,16,54,1686,35,449,3172,444,...,4,2,2,99,99,2,2,398,C*,1
81,0,Greiz,16076,16,55,1654,38,608,2779,563,...,2,2,1,99,99,2,0,399,C*,1


In [108]:
df_final_date = labor_market_by_date.merge(company_changes_date, left_on=['ags2','ags5','date'], right_on=['ags2','ags5','date'])
df_final_date = df_final_date.drop(['Unnamed: 0'], 1)
df_final_date

,Unnamed: 0,ags2,ags5,date,displayed_short_time_work_companies,displayed_short_time_work_people,employees_social_security_at_residence,employees_social_security_at_residenceemployees_social_security_at_work,realized_short_time_work_companies,realized_short_time_work_people,...,number_of_companies_real_estat,number_of_companies_rendering_other_services,number_of_companies_repair_motor_vehicles,number_of_companies_technical_services,number_of_companies_transport,number_of_companies_unknown_sector,number_of_companies_water_and_sewage,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups
0,0,01,01001,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,...,279.0,1089.0,866.0,619.0,105.0,183.0,12.0,15.0,6.0,5.0
1,1,01,01001,2019-02-01,NaN,NaN,NaN,NaN,NaN,NaN,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,11.0,2.0,15.0
2,2,01,01001,2019-03-01,NaN,NaN,44651.0,32144.0,NaN,NaN,...,293.0,568.0,797.0,524.0,103.0,68.0,6.0,6.0,3.0,14.0
3,3,01,01001,2019-04-01,NaN,NaN,NaN,NaN,NaN,NaN,...,207.0,876.0,678.0,404.0,72.0,60.0,9.0,9.0,2.0,14.0
4,4,01,01001,2019-05-01,NaN,NaN,NaN,NaN,NaN,NaN,...,315.0,966.0,923.0,614.0,116.0,46.0,10.0,8.0,2.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11223,11223,16,16077,2020-12-01,54.0,882.0,NaN,NaN,NaN,NaN,...,100.0,1212.0,728.0,385.0,134.0,110.0,26.0,2.0,1.0,6.0
11224,11224,16,16077,2021-01-01,170.0,865.0,NaN,NaN,NaN,NaN,...,99.0,1212.0,729.0,384.0,134.0,110.0,26.0,0.0,3.0,2.0
11225,11225,16,16077,2021-02-01,51.0,252.0,NaN,NaN,NaN,NaN,...,97.0,1210.0,722.0,385.0,134.0,110.0,26.0,0.0,0.0,5.0
11226,11226,16,16077,2021-03-01,16.0,63.0,NaN,NaN,NaN,NaN,...,96.0,1208.0,723.0,388.0,133.0,108.0,26.0,4.0,0.0,4.0


In [109]:
df_final_date

,Unnamed: 0,ags2,ags5,date,displayed_short_time_work_companies,displayed_short_time_work_people,employees_social_security_at_residence,employees_social_security_at_residenceemployees_social_security_at_work,realized_short_time_work_companies,realized_short_time_work_people,...,number_of_companies_real_estat,number_of_companies_rendering_other_services,number_of_companies_repair_motor_vehicles,number_of_companies_technical_services,number_of_companies_transport,number_of_companies_unknown_sector,number_of_companies_water_and_sewage,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups
0,0,01,01001,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,...,279.0,1089.0,866.0,619.0,105.0,183.0,12.0,15.0,6.0,5.0
1,1,01,01001,2019-02-01,NaN,NaN,NaN,NaN,NaN,NaN,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,11.0,2.0,15.0
2,2,01,01001,2019-03-01,NaN,NaN,44651.0,32144.0,NaN,NaN,...,293.0,568.0,797.0,524.0,103.0,68.0,6.0,6.0,3.0,14.0
3,3,01,01001,2019-04-01,NaN,NaN,NaN,NaN,NaN,NaN,...,207.0,876.0,678.0,404.0,72.0,60.0,9.0,9.0,2.0,14.0
4,4,01,01001,2019-05-01,NaN,NaN,NaN,NaN,NaN,NaN,...,315.0,966.0,923.0,614.0,116.0,46.0,10.0,8.0,2.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11223,11223,16,16077,2020-12-01,54.0,882.0,NaN,NaN,NaN,NaN,...,100.0,1212.0,728.0,385.0,134.0,110.0,26.0,2.0,1.0,6.0
11224,11224,16,16077,2021-01-01,170.0,865.0,NaN,NaN,NaN,NaN,...,99.0,1212.0,729.0,384.0,134.0,110.0,26.0,0.0,3.0,2.0
11225,11225,16,16077,2021-02-01,51.0,252.0,NaN,NaN,NaN,NaN,...,97.0,1210.0,722.0,385.0,134.0,110.0,26.0,0.0,0.0,5.0
11226,11226,16,16077,2021-03-01,16.0,63.0,NaN,NaN,NaN,NaN,...,96.0,1208.0,723.0,388.0,133.0,108.0,26.0,4.0,0.0,4.0


In [110]:
#df_final.to_csv('./final_dfs/df_final_with_dates.csv', index = False)